SENTIMENT ANALYSIS ON TELCO

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import nltk
#import ssl
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize 
#nltk.download('punkt')
#nltk.download('sentiwordnet')
#nltk.download('stopwords') 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')



In [2]:
pd.set_option("display.max_colwidth", -1)

C:\Users\lmugi\AppData\Local\Temp/ipykernel_2576/2475433900.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


DATA LOAD

Print out CSV file contains scraped data

In [3]:
df = pd.read_csv('COMBINE DATA GENERAL TRANSLATED.csv')
df.tail()

,Unnamed: 0,id,tweet,date,Label
34302,46219,1.210000e+18,Unifi is already like gomen ... original promo until 31/12/19 ... once extended to 5/2/20. Gomen plaque extend oil subsidies program. Ha ha,Jan-20,Unifi General
34303,46220,1.210000e+18,@helpmeunifi phone line problem @ 0340233199 was reported on 24th Dec 2019 but have not been resolved until today. @CFM_Malaysia,Jan-20,Unifi General
34304,46222,1.210000e+18,Upgrade to UnifiðŸ'¯,Jan-20,Unifi General
34305,46223,1.210000e+18,@helpmeunifi @unifi Unifi TV you want to show this Spartan Ultimate Team Challenge up to a thousand times. … ²Õ²Õ² ..! Move on #unifi #unifitv,Jan-20,Unifi General
34306,46224,1.210000e+18,@NhurAdibah @ashz_26 You chatted about unifi with him. See her reaction later. HA HA HA,Jan-20,Unifi General


In [4]:
#df = df[['Unnamed: 0','tweets','preprocess_tweet']]

DATA CLEANING

Remove Duplicates

In [5]:
df["first_apply"] = (~ df.duplicated ("tweet", keep = "first")). astype (int)
#df = df.set_index("first_apply")
#df = df["first_apply"].drop("0")
df.drop(df.index[df['first_apply'] == 0], inplace = True)

In [6]:
df

,Unnamed: 0,id,tweet,date,Label,first_apply
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1
...,...,...,...,...,...,...
34302,46219,1.210000e+18,Unifi is already like gomen ... original promo until 31/12/19 ... once extended to 5/2/20. Gomen plaque extend oil subsidies program. Ha ha,Jan-20,Unifi General,1
34303,46220,1.210000e+18,@helpmeunifi phone line problem @ 0340233199 was reported on 24th Dec 2019 but have not been resolved until today. @CFM_Malaysia,Jan-20,Unifi General,1
34304,46222,1.210000e+18,Upgrade to UnifiðŸ'¯,Jan-20,Unifi General,1
34305,46223,1.210000e+18,@helpmeunifi @unifi Unifi TV you want to show this Spartan Ultimate Team Challenge up to a thousand times. … ²Õ²Õ² ..! Move on #unifi #unifitv,Jan-20,Unifi General,1


Transform tweets into lower case

In [7]:
df['preprocess_tweet'] = df['tweet'].str.lower()
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi @grabmy. i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸ https://t.co/dtejdvdq4v
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like 97% of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,@digitelco already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,@digitelco hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,@digi_caps looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,@digitelco prepaid can seduction on a ship?
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,@digitelco why your internet is so slow this night haa ??? !!! not only this night but before this also. better you fix it !!!
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,i'm at digi bandar sg. long https://t.co/d1izcmie10
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,"never mind the interior. yesterday i sent it to the customer's house at taman setia perdana, setia alam line only digi 1 bar. urban area as. »â € â € œyou ï¸ https://t.co/prnpsh3qv7"


In [8]:
def remove_num(text):
    text = re.sub('[0-9]', '', text)
    text = text.strip()
    #text = ' '.join(word for word in text.split() if len(word)>3)
    return text
df['preprocess_tweet']= df['preprocess_tweet'].apply(lambda x:remove_num(x))
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi @grabmy. i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸ https://t.co/dtejdvdqv
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like % of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,@digitelco already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,@digitelco hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,@digi_caps looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,@digitelco prepaid can seduction on a ship?
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,@digitelco why your internet is so slow this night haa ??? !!! not only this night but before this also. better you fix it !!!
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,i'm at digi bandar sg. long https://t.co/dizcmie
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,"never mind the interior. yesterday i sent it to the customer's house at taman setia perdana, setia alam line only digi bar. urban area as. »â € â € œyou ï¸ https://t.co/prnpshqv"


Filtering

URLs Removal

In [9]:
def remove_URL(text):
    URL = re.compile(r'https?://\S+|www\.\S+')
    return URL.sub(r'', text)
df['preprocess_tweet'] = df['preprocess_tweet'].apply(remove_URL)
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi @grabmy. i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like % of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,@digitelco already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,@digitelco hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,@digi_caps looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,@digitelco prepaid can seduction on a ship?
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,@digitelco why your internet is so slow this night haa ??? !!! not only this night but before this also. better you fix it !!!
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,i'm at digi bandar sg. long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,"never mind the interior. yesterday i sent it to the customer's house at taman setia perdana, setia alam line only digi bar. urban area as. »â € â € œyou ï¸"


Hashtag Removal

In [10]:
def remove_Hashtag(text):
    Hashtag = re.compile(r'#\S+|www\.\S+')
    return Hashtag.sub(r'', text)
df['preprocess_tweet'] = df['preprocess_tweet'].apply(remove_Hashtag)
df.head(25)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi @grabmy. i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like % of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,@digitelco already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,@digitelco hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,@digi_caps looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,@digitelco prepaid can seduction on a ship?
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,@digitelco why your internet is so slow this night haa ??? !!! not only this night but before this also. better you fix it !!!
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,i'm at digi bandar sg. long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,"never mind the interior. yesterday i sent it to the customer's house at taman setia perdana, setia alam line only digi bar. urban area as. »â € â € œyou ï¸"


Mentions Removal

In [11]:
def remove_Mentions(text):
    Mentions = re.compile(r'@[A-Za-z0-9._/]+')
    return Mentions.sub(r'', text)
df['preprocess_tweet'] = df['preprocess_tweet'].apply(remove_Mentions)
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like % of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,prepaid can seduction on a ship?
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,why your internet is so slow this night haa ??? !!! not only this night but before this also. better you fix it !!!
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,i'm at digi bandar sg. long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,"never mind the interior. yesterday i sent it to the customer's house at taman setia perdana, setia alam line only digi bar. urban area as. »â € â € œyou ï¸"


Emoji Removal

In [12]:
def remove_Emoji(text):
    Emoji = re.compile("["u"\U0001F600-\U0001F64F"u"\U0001F300-\U0001F5FF"u"\U0001F680-\U0001F6FF"u"\U0001F1E0-\U0001F1FF"u"\U00002702-\U000027B0"u"\U000024C2-\U0001F251""]+", flags = re.UNICODE)
    return Emoji.sub(r'', text)
df['preprocess_tweet'] = df['preprocess_tweet'].apply(remove_Emoji)
df.head()

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi i would like to suggest that you can add digi to be in your bill option as per my video recording tq â˜ºï¸
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then .. i dont really need it because im working from home like % of the time so do i really need so much data ??
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,hi digi where are you going? i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didn't know digi to rhb was down now nj


Punctuation Removal

In [13]:
df['preprocess_tweet'] = df['preprocess_tweet'].str.replace('[^\w\s]', '')
df.head(50)

C:\Users\lmugi\AppData\Local\Temp/ipykernel_2576/1294857870.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preprocess_tweet'] = df['preprocess_tweet'].str.replace('[^\w\s]', '')


,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi i would like to suggest that you can add digi to be in your bill option as per my video recording tq âºï
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,was gonna change my digi plan since i changed my job but then i dont really need it because im working from home like of the time so do i really need so much data
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,already dm you reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,hi digi where are you going i want to open ws pon connecting only
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,i didnt know digi to rhb was down now nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,looks so cool with qr code ready to be scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,prepaid can seduction on a ship
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,why your internet is so slow this night haa not only this night but before this also better you fix it
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,im at digi bandar sg long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,never mind the interior yesterday i sent it to the customers house at taman setia perdana setia alam line only digi bar urban area as â â œyou ï


Tokenization:  To split the tweets into words known as tokens

In [14]:
stopwords_nltk = stopwords.words('english')
stopwords_nltk.append('even')
stopwords_nltk.append('ever')
stopwords_nltk.append('etc')
stopwords_nltk.append('still')
stopwords_nltk.append('say')
stopwords_nltk.append('im')
stopwords_nltk.append('tell')
stopwords_nltk.append('thats')

not_stopwords_nltk = {'no', 'nor', 'not','t','can', 'don',"don't",'ain','aren',"aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", }
real_stopwords_nltk = set([word for word in stopwords_nltk if word not in not_stopwords_nltk])
print (real_stopwords_nltk)

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in real_stopwords_nltk])


{'for', 'yourself', 'all', 'any', 've', 'such', 'was', 'having', 'y', 'his', "she's", 'if', 'im', 'but', 'then', 'other', 'i', "it's", 'she', 'when', 'say', 'should', 'it', 'until', 'there', 'ours', "that'll", 'yours', 'on', 'been', 'here', 'because', 'which', 'against', 'that', 'whom', 'ourselves', "you'd", 'they', 'what', 'is', 'into', 'very', 'about', 'm', 'or', 'out', 'again', 'off', 'herself', 'above', 'he', 'own', 'have', 'than', 'while', 'hers', 'etc', 'being', 's', 'yourselves', 'be', "you're", 'few', 'myself', 'who', 'so', 'even', 'these', 'how', 'still', 'tell', 'over', "should've", 'your', 'them', 'before', 'theirs', 'd', 'with', 'further', 'ever', 'an', 'were', 'had', 'by', 'does', 'me', 'of', 'the', 'in', 'has', 'once', 'same', 'a', 'their', 'down', 'himself', 'why', 'each', 'll', 'up', 'too', 'will', 'both', 'her', 'thats', 'below', 'during', 'where', 'most', 'just', 'through', 'to', 'our', 'its', 'him', 'at', 'do', 'as', 'from', 'some', 'am', 'doing', 'this', "you've", '

In [15]:
df["preprocess_tweet"] = df["preprocess_tweet"].apply(stopwords)
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi would like suggest can add digi bill option per video recording tq âºï
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,gonna change digi plan since changed job dont really need working home like time really need much data
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,already dm reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,hi digi going want open ws pon connecting
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,didnt know digi rhb nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,looks cool qr code ready scanned
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,prepaid can seduction ship
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,internet slow night haa not night also better fix
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,digi bandar sg long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,never mind interior yesterday sent customers house taman setia perdana setia alam line digi bar urban area â â œyou ï


In [16]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['preprocess_tweet'] = df['preprocess_tweet'].apply(lambda x: tokenization(x.lower()))
df.head()

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,"[hi, would, like, suggest, can, add, digi, bill, option, per, video, recording, tq, âºï]"
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,"[gonna, change, digi, plan, since, changed, job, dont, really, need, working, home, like, time, really, need, much, data]"
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,"[already, dm, reply]"
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,"[hi, digi, going, want, open, ws, pon, connecting]"
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,"[didnt, know, digi, rhb, nj]"


Normalization - Lemmatization: A process of converting the words to the based form by considering the 
context of the sentiments as a whole. 

In [17]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV} 

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text)
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df['preprocess_tweet'] = df['preprocess_tweet'].apply(lemmatize_words)
df.head(50)

,Unnamed: 0,id,tweet,date,Label,first_apply,preprocess_tweet
0,0,1.540000e+18,Hi @GrabMY. I would like to suggest that you can add digi to be in your bill option as per My video recording tq â˜ºï¸ https://t.co/dTEJDvdq4V,Jun-22,Digi General,1,hi would like suggest can add digi bill option per video record tq âºï
1,1,1.540000e+18,Was gonna change my digi plan since i changed my job but then .. I dont really need it because im working from home like 97% of the time so do i really need so much data ??,Jun-22,Digi General,1,gonna change digi plan since change job dont really need work home like time really need much data
2,2,1.540000e+18,@digitelco already dm you reply,Jun-22,Digi General,1,already dm reply
3,3,1.540000e+18,@digitelco hi digi where are you going? I want to open ws pon connecting only,Jun-22,Digi General,1,hi digi go want open w pon connect
4,4,1.540000e+18,I didn't know digi to rhb was down now nj,Jun-22,Digi General,1,didnt know digi rhb nj
5,5,1.540000e+18,@digi_caps Looks so cool with QR code ready to be scanned,Jun-22,Digi General,1,look cool qr code ready scan
6,6,1.540000e+18,@digitelco prepaid can seduction on a ship?,Jun-22,Digi General,1,prepaid can seduction ship
7,7,1.540000e+18,@digitelco why your internet is so slow this night haa ??? !!! Not only this night but before this also. Better you fix it !!!,Jun-22,Digi General,1,internet slow night haa not night also well fix
8,8,1.540000e+18,I'm at Digi Bandar Sg. Long https://t.co/D1IzcMIe10,Jun-22,Digi General,1,digi bandar sg long
9,9,1.540000e+18,"Never mind the interior. Yesterday I sent it to the customer's house at Taman Setia Perdana, Setia Alam line only digi 1 bar. Urban area as. »Â € â € œYou ï¸ https://t.co/PrNpSh3qV7",Jun-22,Digi General,1,never mind interior yesterday send customer house taman setia perdana setia alam line digi bar urban area â â œyou ï


In [18]:
from collections import Counter
Counter(" ".join(df["preprocess_tweet"]).split()).most_common(200)

[('line', 7691),
 ('unifi', 6038),
 ('want', 4899),
 ('umobile', 4657),
 ('maxi', 4071),
 ('celcom', 3765),
 ('use', 3764),
 ('digi', 3145),
 ('not', 3109),
 ('like', 2973),
 ('can', 2807),
 ('slow', 2701),
 ('no', 2629),
 ('internet', 2432),
 ('get', 2426),
 ('dont', 2374),
 ('problem', 2332),
 ('time', 2253),
 ('please', 1980),
 ('pay', 1901),
 ('â', 1677),
 ('call', 1624),
 ('bad', 1593),
 ('service', 1590),
 ('day', 1577),
 ('really', 1515),
 ('go', 1498),
 ('cant', 1494),
 ('g', 1440),
 ('hi', 1421),
 ('month', 1414),
 ('know', 1310),
 ('change', 1264),
 ('data', 1231),
 ('rm', 1216),
 ('dm', 1174),
 ('bill', 1167),
 ('area', 1157),
 ('speed', 1157),
 ('make', 1148),
 ('right', 1143),
 ('home', 1094),
 ('check', 1057),
 ('already', 1055),
 ('maxis', 1042),
 ('good', 1039),
 ('okay', 1008),
 ('ðÿ', 1007),
 ('house', 971),
 ('ok', 967),
 ('whats', 955),
 ('work', 943),
 ('coverage', 941),
 ('wifi', 939),
 ('today', 921),
 ('phone', 902),
 ('people', 873),
 ('since', 868),
 ('ask', 8

In [19]:
df.to_csv("CLEANED COMBINE DATA GENERAL TRANSLATED.csv")

Stop Words Removal

In [ ]:
print(stopwords_nltk)

In [ ]:
df['preprocess_tweet']= df['preprocess_tweet'].astype(str)

In [ ]:
df.to_csv('CLEANED DATASET TRANSLATED 22_6', encoding='utf-8')

In [ ]:
df["preprocess_tweet"] = df["preprocess_tweet"]

In [ ]:
df['preprocess_tweet']= df['preprocess_tweet'].astype(str)

DATA LABELLING Sentiwordnet

In [7]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\lmugi\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [ ]:
pos=neg=obj=count=0

postagging = []

for review in df['preprocess_tweet']:
    list = word_tokenize(review)
    postagging.append(nltk.pos_tag(list))

df['pos_tags'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    return None

# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wordnet.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []

for pos_val in df['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
df['senti_score'] = senti_score
print(df['senti_score'])

print(df.head)

In [ ]:
def getOverallSentiment(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['Overall Sentiment'] = df['senti_score'].apply(getOverallSentiment)

df[['preprocess_tweet', 'Overall Sentiment']]

In [ ]:
pip install seaborn

In [ ]:
sns.countplot(df['Overall Sentiment'])

Word Cloud

In [ ]:
allWords = ' '.join([twts for twts in df['preprocess_tweet']])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allWords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
def getSentimentLabel(score):
    if score == 'Positive':
        return 1
    elif score == 'Negative':
        return -1
    else:
        return 0

df['Label'] = df['Overall Sentiment'].apply(getSentimentLabel)

df[['Overall Sentiment','Label']]

Score Visualization

In [ ]:
plt.figure(figsize = (8,6))
for i in range(0, df.shape[0]):
    plt.scatter(df['Label'][i], df['senti_score'][i], color = 'Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Label')
plt.ylabel('Overall Sentiment')
plt.show()



Percentage of positive tweets

In [ ]:
positive_tweets = df[df['Overall Sentiment'] == 'Positive']
positive_tweets = positive_tweets['Preprocessed']

round((positive_tweets.shape[0] / df.shape[0]) *100, 1) 

Percentage of negative tweets

In [ ]:
negative_tweets = df[df['Overall Sentiment'] == 'Negative']
negative_tweets = negative_tweets['Preprocessed']

round((negative_tweets.shape[0] / df.shape[0]) *100, 1) 

Percentage of neutral tweets

In [ ]:
neutral_tweets = df[df['Overall Sentiment'] == 'Neutral']
neutral_tweets = neutral_tweets['Preprocessed']

round((neutral_tweets.shape[0] / df.shape[0]) *100, 1) 

Percentage of neutral tweets

Value counts

In [ ]:
df['Overall Sentiment'].value_counts()

#plot and visualize counts

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Overall Sentiment'].value_counts().plot(kind = 'bar')
plt.show()

In [ ]:
X = df['Content']
y =  df['Label']

Training and Testing Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                        test_size = 0.5, random_state=45)

Feature Extraction

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf vectorizer
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
lr_score = lr.score(X_test_vec, y_test)
print("Results for Logistic Regression with tfidf")
print(lr_score)
y_pred_lr = lr.predict(X_test_vec)

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm_knn = confusion_matrix(y_test, y_pred_lr)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lr).ravel()
print(tn, fp, fn, tp)
tpr_knn = round(tp/(tp + fn), 4)
tnr_knn = round(tn/(tn+fp), 4)
print(tpr_knn, tnr_knn)

Support Vector Machine

In [ ]:
from sklearn import svm
#params = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100]}
svcl = svm.SVC(kernel = 'rbf')
#clf_sv = GridSearchCV(svcl, params)
svcl.fit(X_train_vec, y_train)
svcl_score = svcl.score(X_test_vec, y_test)
print("Results for Support Vector Machine with tfidf")
print(svcl_score*100)
y_pred_sv = svcl.predict(X_test_vec)


In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm_sv = confusion_matrix(y_test, y_pred_sv)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_sv).ravel()
print(tn, fp, fn, tp)
tpr_sv = round(tp/(tp + fn), 4)
tnr_sv = round(tn/(tn+fp), 4)
print(tpr_sv, tnr_sv)

Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
nb_score = nb.score(X_test_vec, y_test)
print("Results for Naive Bayes with tfidf")
print(nb_score)
y_pred_nb = nb.predict(X_test_vec) 


In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_nb).ravel()
print(tn, fp, fn, tp)
tpr_nb = round(tp/(tp + fn), 4)
tnr_nb = round(tn/(tn+fp), 4)
print(tpr_nb, tnr_nb)